In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
df = pd.read_excel('Testes_Estatisticos.xlsx', sheet_name='Dados')

In [3]:
df

,Padrão 1,Padrão 2,Padrão 3,Padrão 4,Padrão 5,Padrão 6
0,73439.331429,114422.951429,137847.371429,168777.891429,219072.221429,281650.941429
1,74709.571429,114855.201429,136493.411429,172028.961429,222644.051429,276065.011429
2,75383.991429,116134.251429,135820.571429,172089.501429,222438.491429,277585.041429
3,76085.281429,117401.731429,136135.261429,173948.811429,223538.301429,278630.831429
4,76761.001429,117268.611429,137848.391429,175236.131429,220192.941429,281322.571429
5,76516.321429,117733.621429,138811.591429,175135.401429,220294.161429,280352.981429
6,76067.531429,115657.581429,138185.081429,175373.971429,220751.101429,281291.211429


In [4]:
# Teste de normalidade (Shapiro Wilk)
stat, pvalue = stats.shapiro(df['Padrão 1'])
if pvalue < 0.05:
    print('Os dados não são normalmente distribuídos.')
else:
    print('Os dados são normalmente distribuídos.')

# Teste de homocedasticidade (Cochran)
stat, pvalue = stats.levene(df['Padrão 1'], df['Padrão 1'])
if pvalue < 0.05:
    print('Os dados não são homocedásticos.')
else:
    print('Os dados são homocedásticos.')

# ANOVA (falta de ajuste e erro puro)
F, pvalue = stats.f_oneway(df['Padrão 1'], df['Padrão 1'])
if pvalue < 0.05:
    print('Existe uma diferença significativa entre os grupos.')
else:
    print('Não existe uma diferença significativa entre os grupos.')

Os dados são normalmente distribuídos.
Os dados são homocedásticos.
Não existe uma diferença significativa entre os grupos.


In [5]:
import statsmodels.stats.stattools as stattools

# Importar os dados do Excel
df = pd.read_excel('Testes_Estatisticos.xlsx', sheet_name='Dados')

# Obter o teste de Durbin-Watson
dw = stattools.durbin_watson(df['Padrão 1'], df['Padrão 2'], df['Padrão 3'], df['Padrão 4'], df['Padrão 5'], df['Padrão 6'])

# Imprimir o resultado do teste
print(dw)

TypeError: durbin_watson() takes from 1 to 2 positional arguments but 6 were given

In [ ]:
import scipy.stats.mstats as mstats

# Importar os dados do Excel
df = pd.read_excel('Testes_Estatisticos.xlsx', sheet_name='Dados')

# Obter o teste de Grubbs
g = mstats.grubbs(df['Padrão 1'], df['Padrão 2'], df['Padrão 3'], df['Padrão 4'], df['Padrão 5'], df['Padrão 6'])

# Imprimir o resultado do teste
print(g)